In [ ]:
#ROUNDABOUT SIMULATION

#GLOBAL VARIABLES

N = "North"
S = "South"
E = "East"
W = "West"
MEAN_ARRIVAL_TIME = 10
PRINT_EVENTS = True

class Car:
    
    #each car has its own arrival time, arrival direction, entrance speed, destination
    def __init__(self, arrival_time, arr_dir, dest, speed):
        self.arrival_time = arrival_time
        self.arr_dir = arr_dir
        self.dest = dest
        self.speed = speed
        
    #def 
    

class Pedestrian:

    #each pedestrian has its own arrival time, destination, and time to cross street
    def __init__(self, arrival_time, dest, cross_time):
        self.arrival_time = arrival_time
        self.dest = dest
        self.cross_time = cross_time

class Event:

    def __init__(self, event_type, time, dest):
        self.type = event_type
        self.time = time
        self.dest = dest

class Simulation:

    upper_arrival_time = 2 * MEAN_ARRIVAL_TIME

    def __init__(self, total_arrivals):
        self.arrivals = 0
        self.total_arrivals = total_arrivals
        self.clock = 0

        #cars waiting to enter roundabout
        self.north, self.east, self.south, self.west = [], [], [], []
        #queues in roundabout
        self.left_straight, self.right = [], []

        #enables us to see what is going on
        self.print_events = PRINT_EVENTS
        #saves our data for arrival time of 4*3*2 + 4 = 28 cases
        self.self_driv = [[0]*3]*4      #self driving data
        self.humn_driv = [[0]*3]*4      #human driver data
        self.ped = [0]*4                #pedestrian data

    def run(self):
        while self.arrivals <= self.total_arrivals:
            if self.print_events:
                self.print_state()
            self.execute_next_event()
